# Lab Assignment Two: Exploring Image Data

***Christopher Cook, Bonita Davis, Anekah Kelley, Davis Lynn***

## 1. Preparation and Overview

### **1.1 Business Understanding**

This is a dataset of 15,149 songs from 3,083 artists over the course of over a century (1899-2024).  It is sorted by 18 different features:
*    Track (string)
*    Artist (string)
*    Year (int)
*    Duration (int, in ms)
*    Time Signature (int, numerator of time signature; i.e. 4 means 4/4)
*    Danceability (float, 0-10)
*    Energy (float 0-1)
*    Key (int, 0-11; 0 = C, etc.)
*    Loudness (float, loudness in db -47.4=0.92)
*    Mode (int, major/minor modality; 0/1)
*    Speechiness (float, 0-1)
*    Acousticness (float, 0-1)
*    Instrumentalness (float 0-1)
*    Liveness (float, 0-1)
*    Valence (float, 0-1)
*    Tempo (tempo, BPM from 0-220)
*    Popularity (int, 0-100)
*    Genre (string)

This data was collected directly from the Spotify API, allowing for additional attributes such as Loudness, Speechiness, etc. The dataset is free to download on Kaggle through the link below.

Data Source: https://www.kaggle.com/datasets/thebumpkin/10400-classic-hits-10-genres-1923-to-2023/data

The task of this dataset is to serve as a touchpoint for individuals in the music industry to explore trends, compare genres, analyze sonic qualities, and see how classic hits across time periods and genres compare to once another (Kaggle).  The sheer size of the dataset, along with the amount of information provided about each song, makes this an excellent resource to understand musical trends.  It can also be used to see how an individual artist or genre has performed over time, providing insights into how future songs/albums might perform.

This is the primary business-case of this dataset and our classifier -- attempting to predict attributes about songs.  This information can be used by companies like Spotify and Apple Music to predict the likelihood that a user will like a song based on the attributes of the songs they have been previously shown to like (if they typically like pop music with a loudness of 97, that information can be used to predict new songs that they may also like).  These businesses care more about the relationships between all of the attributes, as it is the combination of these attributes that creates a listening style.  This information could also be used by a music producer or musician to determine what attributes about songs in different genres/time periods tend to make them popular in order to see how to best leverage the artist's skills to create a hit song.  These businesses care more about the interplay between the attributes that they deem to be relevant based on the artist and popularity, as that gives the best blueprint for success.

This model would likely be deployed, although it may differ depending on the user.  Users like Spotify, Apple Music, YouTube Music, and other such companies would use this data constantly, making it a deployed model.  In contrast, while some music producers may consistently run new models, it is possible that it could be semi-offline and just generated once for the relevant artists.  This data was likely collected with these intentions in mind, as classifiers could be trained and sold to such businesses for a sizeable amount of money.

The use of such a classifier by companies like Spotify and Apple Music is supported by the fact that Spotify currently uses a similar classifier to create their "Made For You" playlist provided to users (https://www.music-tomorrow.com/blog/how-spotify-recommendation-system-works-a-complete-guide-2022).  They use very similar attributes (which is the case because our dataset directly incorporates Spotify information) to predict the likelihood that a user will enjoy a song based on their listening history.  Spotify's algorithm is many times more complex than anything we would be capable of creating at this point in time, so holding ourselves to that standard would be simply foolish.  Additionally, Spotify does not publish an "accuracy" rate for predicting songs.  Further complicating the problem, an NIH study showed that roughly 15% of people skip songs that they haven't listened to, before even listening to it, and that others skipped the songs shortly after starting them if they didn't quickly enjoy the song (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7004375/).  While this is a relatively small percentage, it shows the importance of a very accurate classifier.  Ideally, a classifier with a roughly 90% accuracy is likely possible with advanced algorithms, predicts songs with an acceptable margin of error, and will result in benefits for companies.

For music producers and artists interested in predicting the success of their songs, a much lower bar of accuracy is acceptable.  Similar classifier algorithms boast an accuracy of anywhere between 55-77% (https://cs229.stanford.edu/proj2015/140_report.pdf, https://towardsdatascience.com/song-popularity-predictor-1ef69735e380, https://www.nature.com/articles/s41598-022-25430-9).  In truth, anything able to predict with an accuracy greater than 50% would be considered successful to some degree, as popularity is highly subjective, changes dramatically over time, and is often a product of purely qualitative features that are difficult to connect to quantitative information, despite best efforts.  Aiming for an accuracy of roughly 65% would put our model squarely in the middle of currently available algorithms, making it both realistic and acceptable in the industry.

### 1.2 Preparation of Data

### 1.3 Training/Testing Data Split

## 2. Modeling

In [5]:
import numpy as np
from scipy.special import expit
from sklearn.model_selection import train_test_split
import pandas as pd
import plotly
from sklearn.metrics import accuracy_score

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
class BinaryLogisticRegressionBase:
    # private:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        return 'Base Binary Logistic Regression Object, Not Trainable'
    
    # convenience, private and static:
    @staticmethod
    def _sigmoid(theta):
        return 1/(1+np.exp(-theta)) 
    
    @staticmethod
    def _add_intercept(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term
    
    # public:
    def predict_proba(self, X, add_intercept=True):
        # add bias term if requested
        Xb = self._add_intercept(X) if add_intercept else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X):
        return (self.predict_proba(X)>0.5) #return the actual prediction

In [ ]:
class BinaryLogisticRegression(BinaryLogisticRegressionBase):
    #private:
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'


    @property
    def coef_(self):
        if(hasattr(self,'w_')):
            return self.w_[1:]
        else:
            return None

    @property
    def intercept_(self):
        if(hasattr(self,'w_')):
            return self.w_[0]
        else:
            return None

        
    def _get_gradient(self,X,y):
        # programming \sum_i (yi-g(xi))xi
        gradient = np.zeros(self.w_.shape) # set gradient to zero
        for (xi,yi) in zip(X,y):
            # the actual update inside of sum
            gradi = (yi - self.predict_proba(xi,add_intercept=False))*xi 
            # reshape to be column vector and add to gradient
            gradient += gradi.reshape(self.w_.shape) 
        
        return gradient/float(len(y))
       
    # public:
    def fit(self, X, y):
        Xb = self._add_intercept(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate 

In [ ]:
class VectorBinaryLogisticRegression(BinaryLogisticRegression):
    # inherit from our previous class to get same functionality
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    
    # but overwrite the gradient calculation
    def _get_gradient(self, X, y):
        # Ensure y is numeric, converting categorical to integers or floats
        y = y.astype('float')  # or y = y.astype('int') if appropriate
    
        ydiff = y - self.predict_proba(X, add_intercept=False).ravel()  # Get y difference
        ydiff = np.array(ydiff)  # Convert ydiff to a NumPy array to allow multi-dimensional indexing
        gradient = np.mean(X * ydiff[:, np.newaxis], axis=0)  # Multiply ydiff as a column vector

        return gradient.reshape(self.w_.shape)

In [ ]:
class LogisticRegression:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'

    @property
    def coef_(self):
        if(hasattr(self,'w_')):
            return self.w_[:,1:]
        else:
            return None

    @property
    def intercept_(self):
        if(hasattr(self,'w_')):
            return self.w_[:,0]
        else:
            return None
        
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = (y==yval) # create a binary problem
            # train the binary classifier for this class
            blr = VectorBinaryLogisticRegression(self.eta,
                                                 self.iters)
            blr.fit(X,y_binary)
            # add the trained classifier to the list
            self.classifiers_.append(blr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
        
    def predict_proba(self,X):
        probs = []
        for blr in self.classifiers_:
            probs.append(blr.predict_proba(X)) # get probability for each classifier
        
        return np.hstack(probs) # make into single matrix
    
    def predict(self,X):
        return self.unique_[np.argmax(self.predict_proba(X),axis=1)] # take argmax along row

In [ ]:
df = pd.read_csv('ClassicHit.csv')

#This is how I (Chris) believe the train/test data should be done

Xcoloumns = ['Year', 'Duration', 'Time_Signature', 'Danceability', 'Energy', 'Key', 'Loudness', 'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo']

X = df[Xcoloumns]
y = pd.cut(df['Popularity'], bins=[-float('inf'), 15, 75, float('inf')], labels=[0, 1, 2]) #I am choosing the splits to determine whether a song will be, not populary at all, mediocre, very popular
#I chose this way so we can find something meaningful no matter what the classification is for the songs
#y = (df['Popularity'] > 75).astype(int) #Testing for binary case

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size=0.2)

plotly.offline.init_notebook_mode() # run at the start of every notebook

graph1 = {'x': np.unique(y),
          'y': np.bincount(y)/len(y),
            'type': 'bar'}

fig = dict()
fig['data'] = [graph1]
fig['layout'] = {'title': 'Class Distribution',
                'autosize':False,
                'width':400,
                'height':400}

plotly.offline.iplot(fig)

In [ ]:
lr = LogisticRegression(0.1,500)
lr.fit(X_train,y_train)
print(lr)

yhat = lr.predict(X_test)
print('Accuracy of: ',accuracy_score(y_test,yhat))

## 3. Deployment

## 4. Exceptional Work